In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
from functions import pathfile
import pandas as pd

pf=pathfile()
#autenticacion
credentials=service_account.Credentials.from_service_account_file(pf)

project_id="bigquery-416916"

client=bigquery.Client(credentials=credentials, project=project_id)



C:\Users\ingva\OneDrive\Documents\EstudioML\data\raw\bigquery-416916-a0e3871b9a07.json


In [2]:
#Seleccion del data set
daset_ref=client.dataset('data_estudio', project="bigquery-416916")
dataset=client.get_dataset(daset_ref)

In [3]:
#Cuantas Yablas hay en el Data Set
tables=list(client.list_tables(dataset))
for i in tables:
    print(i.table_id)

crime


In [4]:
#Cuantas Tablas hay en el data set
len(tables)

1

In [5]:
table_ref=daset_ref.table('crime')
table=client.get_table(table_ref)
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', None, None, (), None),
 SchemaField('case_number', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('block', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('iucr', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('primary_type', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('location_description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('arrest', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('domestic', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('beat', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('district', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('ward', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('community_area', 'INTEGER', 'NULLABLE', None, None, (), None),
 

In [6]:

df=client.list_rows(table, max_results=100).to_dataframe()

In [7]:
df.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11511120,JB521056,2018-11-18 07:55:00+00:00,0000X E LAKE ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,STREET,False,False,...,42,32,02,1176916.0,1901744.0,2018,2018-11-25 04:09:36+00:00,41.885729,-87.625781,"(41.885729338, -87.625780935)"
1,3255033,HK280478,2004-04-03 04:24:00+00:00,005XX W MADISON ST,031A,ROBBERY,ARMED: HANDGUN,ALLEY,False,False,...,42,28,03,1173039.0,1900311.0,2004,2018-02-28 03:56:25+00:00,41.881884,-87.640060,"(41.881883933, -87.640060441)"
2,4759802,HM369459,2006-05-24 03:50:00+00:00,003XX S JEFFERSON ST,031A,ROBBERY,ARMED: HANDGUN,STREET,False,False,...,2,28,03,1172380.0,1898775.0,2006,2018-02-28 03:56:25+00:00,41.877684,-87.642526,"(41.87768363, -87.642525656)"
3,9914893,HY104370,2015-01-05 05:10:00+00:00,0000X E LAKE ST,0320,ROBBERY,STRONGARM - NO WEAPON,SIDEWALK,False,False,...,42,32,03,1176916.0,1901744.0,2015,2018-02-10 03:50:01+00:00,41.885729,-87.625781,"(41.885729338, -87.625780935)"
4,8651681,HV327109,2012-06-09 10:20:00+00:00,001XX N STATE ST,0320,ROBBERY,STRONGARM - NO WEAPON,CTA TRAIN,False,False,...,42,32,03,1176330.0,1901649.0,2012,2016-02-04 06:33:39+00:00,41.885482,-87.627936,"(41.885481891, -87.627935689)"


In [8]:
df.iloc[:,0]

0     11511120
1      3255033
2      4759802
3      9914893
4      8651681
        ...   
95     9443572
96    11180146
97     8370939
98    11865166
99    12083475
Name: unique_key, Length: 100, dtype: Int64

In [9]:
#instrucciones sql

query="""
      SELECT *
      FROM `bigquery-416916.data_estudio.crime`
      WHERE primary_type ='ROBBERY'
      LIMIT 10
      """
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query_job = client.query(query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))
ONE_MB = 1000*1000
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)
safe_query_job = client.query(query)

df2=safe_query_job.to_dataframe()
df2.head()

This query will process 1690497546 bytes.


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,7791632,HS600251,2010-11-04 11:30:00+00:00,001XX S RIVERSIDE PLZ,0313,ROBBERY,ARMED: OTHER DANGEROUS WEAPON,STREET,False,False,...,2,28,03,1173421.0,1899683.0,2010,2018-02-10 03:50:01+00:00,41.880152,-87.638676,"(41.880152185, -87.638676422)"
1,6029749,HP132911,2008-01-19 05:09:42+00:00,001XX S CLINTON ST,031A,ROBBERY,ARMED: HANDGUN,CONVENIENCE STORE,False,False,...,2,28,03,1172749.0,1899713.0,2008,2018-02-28 03:56:25+00:00,41.880249,-87.641143,"(41.880249406, -87.641143018)"
2,5034855,HM643139,2006-10-06 03:04:00+00:00,005XX W LAKE ST,031A,ROBBERY,ARMED: HANDGUN,GROCERY FOOD STORE,False,False,...,42,28,03,1172491.0,1901698.0,2006,2018-02-28 03:56:25+00:00,41.885702,-87.642032,"(41.885702079, -87.642031644)"
3,10045642,HY233179,2015-04-23 04:50:00+00:00,001XX N STATE ST,031A,ROBBERY,ARMED: HANDGUN,CTA PLATFORM,False,False,...,42,32,03,1176331.0,1901632.0,2015,2018-02-10 03:50:01+00:00,41.885435,-87.627933,"(41.88543522, -87.62793253)"
4,11532609,JB549532,2018-12-11 07:45:00+00:00,001XX N STATE ST,031A,ROBBERY,ARMED: HANDGUN,DEPARTMENT STORE,True,False,...,42,32,03,1176352.0,1900927.0,2018,2019-01-23 11:23:07+00:00,41.883500,-87.627877,"(41.883500187, -87.627876698)"


In [16]:
query=""" 
        SELECT primary_type as Tipo, count(description) as Casos
        FROM `bigquery-416916.data_estudio.crime`
        GROUP BY Tipo
        HAVING Casos>10
        
        """
q_r=client.query(query)
df=q_r.to_dataframe()
df.head()


,Tipo,Casos
0,CRIM SEXUAL ASSAULT,27522
1,ROBBERY,302123
2,SEX OFFENSE,31978
3,NARCOTICS,752390
4,INTERFERENCE WITH PUBLIC OFFICER,18885


In [11]:
df.shape

(34, 2)

In [18]:
query="""
        SELECT year as Year, count(case_number) as Num_Cases
        FROM `bigquery-416916.data_estudio.crime`
        GROUP BY Year
        ORDER BY Year
        

"""
query_df=client.query(query)
df_3=query_df.to_dataframe()
df_3.head()

,Year,Num_Cases
0,2001,485904
1,2002,486815
2,2003,475987
3,2004,469428
4,2005,453777
